# DS-SF-34 | 03 | Databases, Scrapping, and APIs | Codealong | Starter Code

# Part A | RDBMS Databases and `SQLite`

In [1]:
import os

import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

import sqlite3

> ### How to connect to a `SQLite` database

In [2]:
db = sqlite3.connect(os.path.join('..', 'datasets', 'dataset-03-zillow.db'))

> ### How to use `pandas` to run SQL queries and get the results as a `DataFrame`

In [3]:
# The ''' allows for use of multipe lines for strings pd.io.sql.read_sql("SELECT * FROM properties LIMIT 10", con = db)
pd.io.sql.read_sql(
'''
SELECT *
    FROM properties
    LIMIT 10
;
''', con = db)
# Select all columns from (name of table) and limit to first (number) rows. ; is used to stop statement

,id,address,latitude,longitude,is_a_studio,...,size,size_unit,lot_size,lot_size_unit,built_in_year
0,15063471,"55 Vandewater St APT 9, San Francisco, CA",37805103,-122412856,0,...,550.0,sqft,NaN,None,1980.0
1,15063505,"740 Francisco St, San Francisco, CA",37804420,-122417389,0,...,1430.0,sqft,2435.0,sqft,1948.0
2,15063609,"819 Francisco St, San Francisco, CA",37803728,-122419055,0,...,2040.0,sqft,3920.0,sqft,1976.0
3,15064044,"199 Chestnut St APT 5, San Francisco, CA",37804392,-122406590,0,...,1060.0,sqft,NaN,None,1930.0
4,15064257,"111 Chestnut St APT 403, San Francisco, CA",37804240,-122405509,0,...,1299.0,sqft,NaN,None,1993.0
5,15064295,"111 Chestnut St APT 702, San Francisco, CA",37804240,-122405509,0,...,1033.0,sqft,NaN,None,1993.0
6,15064391,"1821 Grant Ave APT 101, San Francisco, CA",37803748,-122408531,0,...,1048.0,sqft,NaN,None,1975.0
7,15064536,"2300 Leavenworth St, San Francisco, CA",37802408,-122417537,0,...,2115.0,sqft,1271.0,sqft,1913.0
8,15064640,"1047-1049 Lombard St, San Francisco, CA",37801889,-122418704,1,...,4102.0,sqft,3049.0,sqft,1948.0
9,15064669,"1055 Lombard St # C, San Francisco, CA",37801873,-122418834,0,...,1182.0,sqft,NaN,None,1986.0


### `SELECT`

In [4]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price, sale_price_unit
    FROM transactions
;
''', con = db)

,id,sale_price,sale_price_unit
0,15063471,710000.00,$
1,15063505,2.15,$M
2,15063609,5.60,$M
3,15064044,1.50,$M
4,15064257,970000.00,$
...,...,...,...
995,2124214951,390000.00,$
996,2126960082,860000.00,$
997,2128308939,830000.00,$
998,2131957929,835000.00,$


### `WHERE`

In [8]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price 
    FROM transactions
    WHERE sale_price_unit = '$M'
;
''', con = db)
# WHERE finds applicable conditions and pulls those rows

,id,sale_price
0,15063505,2.15
1,15063609,5.60
2,15064044,1.50
3,15064536,2.83
4,15064640,4.05
...,...,...
570,2106229715,1.24
571,2108411930,1.73
572,2108546010,1.40
573,2121221385,1.43


In [9]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price / 1000000 AS sale_price
    FROM transactions
    WHERE sale_price_unit = '$'
;
''', con = db)

,id,sale_price
0,15063471,0.710
1,15064257,0.970
2,15064295,0.940
3,15064391,0.835
4,15065032,0.800
...,...,...
420,2124214951,0.390
421,2126960082,0.860
422,2128308939,0.830
423,2131957929,0.835


> #### Here's let's convert the unit of `sale_price` from \$ to \$M

In [11]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price / 1000000 AS sale_price
    FROM transactions
    WHERE sale_price_unit = '$'
    
UNION ALL

SELECT id, sale_price 
    FROM transactions
    WHERE sale_price_unit = '$M'
;
''', con = db)

,id,sale_price
0,15063471,0.710
1,15064257,0.970
2,15064295,0.940
3,15064391,0.835
4,15065032,0.800
...,...,...
995,2106229715,1.240
996,2108411930,1.730
997,2108546010,1.400
998,2121221385,1.430


### `UNION`

> #### Let's combine the previous two queries into one.  `sale_price` is now in \$M accross the dataset

In [12]:
pd.io.sql.read_sql('''
SELECT id, sale_price / 1000000 AS sale_price
    FROM transactions
    WHERE sale_price_unit = '$'
UNION ALL
SELECT id, sale_price
    FROM transactions
    WHERE sale_price_unit = '$M'
;
''', con = db)

,id,sale_price
0,15063471,0.710
1,15064257,0.970
2,15064295,0.940
3,15064391,0.835
4,15065032,0.800
...,...,...
995,2106229715,1.240
996,2108411930,1.730
997,2108546010,1.400
998,2121221385,1.430


### `WITH`

In [13]:
# This is an "In query ..." where the variable only exists within that query
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M')

SELECT *
    FROM normalized_sale_prices
;
''', con = db)

,id,sale_price
0,15063471,0.710
1,15064257,0.970
2,15064295,0.940
3,15064391,0.835
4,15065032,0.800
...,...,...
995,2106229715,1.240
996,2108411930,1.730
997,2108546010,1.400
998,2121221385,1.430


### `JOIN`

In [14]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M')

SELECT properties.beds AS beds, normalized_sale_prices.sale_price AS sale_price
    FROM properties
    JOIN normalized_sale_prices
        ON normalized_sale_prices.id = properties.id
;
''', con = db)

,beds,sale_price
0,1.0,0.710
1,2.0,0.970
2,2.0,0.940
3,1.0,0.835
4,1.0,0.800
...,...,...
995,3.0,1.240
996,2.0,1.730
997,2.0,1.400
998,NaN,1.430


### `WITH` (take 2)

In [15]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

beds_normalized_sale_prices AS
    (SELECT properties.beds AS beds, normalized_sale_prices.sale_price AS sale_price
        FROM properties
        JOIN normalized_sale_prices
            ON normalized_sale_prices.id = properties.id)

SELECT *
    FROM beds_normalized_sale_prices
;
''', con = db)

,beds,sale_price
0,1.0,0.710
1,2.0,0.970
2,2.0,0.940
3,1.0,0.835
4,1.0,0.800
...,...,...
995,3.0,1.240
996,2.0,1.730
997,2.0,1.400
998,NaN,1.430


### `GROUP BY` and `AVG`

> ### Let's now compute the average sale price of these properties based on their number of bedrooms

In [27]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

beds_normalized_sale_prices AS
    (SELECT properties.beds AS beds, normalized_sale_prices.sale_price AS sale_price
        FROM properties
        JOIN normalized_sale_prices
            ON normalized_sale_prices.id = properties.id)

SELECT AVG(sale_price) AS mean_sale_price
    FROM beds_normalized_sale_prices
    GROUP BY beds
;
''', con = db)

,mean_sale_price
0,1.494858
1,0.820425
2,1.192398
3,1.404323
4,1.879330
5,2.867720
6,2.587222
7,5.543167
8,1.960000
9,1.766667


Note: `beds = 0` usually refers to studios but not always...

> ### Activity
> #### Compute the average sales price of studios

In [46]:
# TODO
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

beds_normalized_sale_prices AS
    (SELECT properties.beds AS beds, normalized_sale_prices.sale_price AS sale_price, properties.is_a_studio AS studio
        FROM properties
        JOIN normalized_sale_prices
            ON normalized_sale_prices.id = properties.id)

SELECT AVG(sale_price)
    FROM beds_normalized_sale_prices
    WHERE studio = 1
    
;
''', con = db)

,AVG(sale_price)
0,1.464034


> ### Activity
> #### Find the properties in the dataset that have different addresses but the same latitude/longitude.  Return the smallest set of pair of properties (`address_a`, `address_b`)

In [49]:
# TODO Classmate's code
pd.io.sql.read_sql(
'''
WITH same_lat_long AS
   (select latitude, longitude, count(*) as num_with_latlong
   from properties
   group by latitude, longitude
   having num_with_latlong > 1
   ),
  
   lat_long_groups AS
   (select a.latitude, a.longitude, address, id
   from properties a
   join same_lat_long b
   on a.latitude=b.latitude and a.longitude=b.longitude
   )
   
select a.latitude, a.longitude, a.address as address1, b.address as address2 
from lat_long_groups a
   join lat_long_groups b
   on a.latitude=b.latitude and a.longitude=b.longitude
   where a.id <> b.id 
order by a.latitude, a.longitude, a.address
;
''', con=db)

,a.latitude,a.longitude,address1,address2
0,37710480,-122467645,"3981 Alemany Blvd APT 206, San Francisco, CA","8300 Oceanview Ter APT 306, San Francisco, CA"
1,37710480,-122467645,"3981 Alemany Blvd APT 206, San Francisco, CA","8400 Oceanview Ter APT 218, San Francisco, CA"
2,37710480,-122467645,"8300 Oceanview Ter APT 306, San Francisco, CA","3981 Alemany Blvd APT 206, San Francisco, CA"
3,37710480,-122467645,"8300 Oceanview Ter APT 306, San Francisco, CA","8400 Oceanview Ter APT 218, San Francisco, CA"
4,37710480,-122467645,"8400 Oceanview Ter APT 218, San Francisco, CA","3981 Alemany Blvd APT 206, San Francisco, CA"
...,...,...,...,...
211,37800573,-122424849,"2701 Van Ness Ave APT 702, San Francisco, CA","2701 Van Ness Ave APT 207, San Francisco, CA"
212,37804240,-122405509,"111 Chestnut St APT 403, San Francisco, CA","111 Chestnut St APT 702, San Francisco, CA"
213,37804240,-122405509,"111 Chestnut St APT 702, San Francisco, CA","111 Chestnut St APT 403, San Francisco, CA"
214,37804334,-122412450,"445 Francisco St APT 304, San Francisco, CA","530 Chestnut St APT 407, San Francisco, CA"


In [55]:
pd.io.sql.read_sql(
'''
WITH id_lat_long AS (SELECT id, latitude, longitude
    FROM properties)
    
SELECT a.id AS a_id, b.id AS b_id
    FROM id_lat_long AS a, id_lat_long as b
    WHERE (a.latitude = b.latitude) AND (a.longitude = b.longitude) AND (a.id < b.id)
    
;
''', con=db)
# Compare table with itself under different names. Make sure each pair is represented only once by making only the
# smaller a's valid

,a_id,b_id
0,15064257,15064295
1,15066001,15066036
2,15067583,15067604
3,15067890,15067896
4,15068514,15068551
...,...,...
103,124397844,124397860
104,124850278,124862135
105,125156575,2100850402
106,2100643449,2100786271
